In [1]:
from distribution_fit_class import DistributionFit
from portfolio_optimization_class import PortfolioOptimization, optimize_windows
import pandas as pd
import numpy as np

In [2]:
distribution_fit = DistributionFit()

file_path = r'data\random_data\1_stocks_per_sector_1_iter.csv'
distribution_fit.load_df_from_csv(file_path)

returns_df = pd.read_csv(file_path, index_col=0, parse_dates=True)
returns_df.columns

Index(['MMM', 'A', 'SNPS', 'OMC', 'ROST', 'EIX', 'GS', 'NUE', 'PSA', 'GIS',
       'WMB'],
      dtype='object')

In [3]:
window_size = 32
taus = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.15, 0.2]
maximum_weight = 0.3

In [ ]:
tau_aggregation = []
tau_dfs = []

for tau_index, tau in enumerate(taus):
    tau_dict = {}
    portfolios = optimize_windows(number_of_quarters=window_size, tau=tau, maximum_weight=maximum_weight, data_path=file_path)

    expected_validation_values_norm = []
    expected_scoring_values_norm = []
    expected_validation_values_t = []
    expected_scoring_values_t = []
    for window in range(0, 64):
        if window % 5 == 0:
            print(f'Tau: {tau}, window - {window}')
        returns_window = returns_df.iloc[window : window + window_size]
        distribution_fit = DistributionFit()
        distribution_fit.set_df(returns_window)
        distribution_fit.fit_distribution_all_stocks()

        simulated_data_path = r'data\copulas_outputs\clayton_random_fixed_coefficient.csv'

        df = pd.read_csv(simulated_data_path)
        df.columns = returns_df.columns          # RENAME OF COLUMNS CAUSE THEY GOT MIXED UP IN ANALYSIS IN R !!!!!!!!!!!!!!!!
        # df.to_csv(simulated_data_path, index=False)
        distribution_fit.set_simulated_data_from_df(df.iloc[:1000])

        distribution_fit.calculate_returns_from_simulated_quantiles()
        returns_norm = distribution_fit.get_simulated_return_norm()
        returns_t = distribution_fit.get_simulated_return_t_student()
        weights = portfolios.iloc[window].filter(regex="^w\d+$").values

        expectile = -portfolios.iloc[window]["EVAR"]


        portfolio_returns_norm = np.sum(returns_norm.values * weights, axis=1)  # - 0.2 to check whether validation values can be positive
        validation_values_norm = (1 - tau) * (-1) * np.minimum(portfolio_returns_norm - expectile, 0) - tau * np.maximum(portfolio_returns_norm - expectile, 0)
        # in scoring fun negative part will be turned into a positive one by the power of 2 anyway
        scoring_values_norm = (1 - tau) * np.minimum((portfolio_returns_norm - expectile)**2, 0) + tau * np.maximum((portfolio_returns_norm - expectile)**2, 0)
        expected_validation_values_norm.append(np.mean(validation_values_norm))
        expected_scoring_values_norm.append(np.mean(scoring_values_norm))

        portfolio_returns_t = np.sum(returns_t.values * weights, axis=1)
        validation_values_t = (1 - tau) * (-1) * np.minimum(portfolio_returns_t - expectile, 0) - tau * np.maximum(portfolio_returns_t - expectile, 0)
        # in scoring fun negative part will be turned into a positive one by the power of 2 anyway
        scoring_values_t = (1 - tau) * np.minimum((portfolio_returns_t - expectile)**2, 0) + tau * np.maximum((portfolio_returns_t - expectile)**2, 0)
        expected_validation_values_t.append(np.mean(validation_values_t))
        expected_scoring_values_t.append(np.mean(scoring_values_t))

    tau_dict[f'clayton+gauss_dist+val'] = expected_validation_values_norm
    tau_dict[f'clayton+gauss_dist+score'] = expected_scoring_values_norm
    tau_dict[f'clayton+t_dist+val'] = expected_validation_values_t
    tau_dict[f'clayton+t_dist+scor'] = expected_scoring_values_t

    tau_aggregation.append(tau_dict)
    df = pd.DataFrame.from_dict(tau_dict)
    tau_dfs.append(df)

Tau: 0.0005, window - 0
Tau: 0.0005, window - 5
Tau: 0.0005, window - 10
Tau: 0.0005, window - 15
Tau: 0.0005, window - 20
Tau: 0.0005, window - 25
Tau: 0.0005, window - 30
Tau: 0.0005, window - 35
Tau: 0.0005, window - 40
Tau: 0.0005, window - 45
Tau: 0.0005, window - 50
Tau: 0.0005, window - 55
Tau: 0.0005, window - 60
Tau: 0.001, window - 0
Tau: 0.001, window - 5
Tau: 0.001, window - 10
Tau: 0.001, window - 15
Tau: 0.001, window - 20
Tau: 0.001, window - 25
Tau: 0.001, window - 30
Tau: 0.001, window - 35
Tau: 0.001, window - 40
Tau: 0.001, window - 45
Tau: 0.001, window - 50
Tau: 0.001, window - 55
Tau: 0.001, window - 60
Tau: 0.002, window - 0
Tau: 0.002, window - 5
Tau: 0.002, window - 10
Tau: 0.002, window - 15
Tau: 0.002, window - 20
Tau: 0.002, window - 25
Tau: 0.002, window - 30
Tau: 0.002, window - 35
Tau: 0.002, window - 40
Tau: 0.002, window - 45
Tau: 0.002, window - 50
Tau: 0.002, window - 55
Tau: 0.002, window - 60
Tau: 0.005, window - 0
Tau: 0.005, window - 5
Tau: 0.005,

In [28]:
tau_aggregation[0]

{'clayton+gauss_dist+val': [0.0013341189181943095,
  0.00039492817144567044,
  0.00036755876684329544,
  9.27199541593134e-05,
  0.00015643030150087325,
  0.0006024839807688979,
  0.000524339944519551,
  0.0005505044860883242,
  0.0003679365930969299,
  0.000396441786436207,
  0.0003970366724276017,
  0.00026964555319827977,
  0.0006428250870722281,
  0.0006698081683405803,
  0.0009606547360062446,
  0.0010929130507487097,
  0.0004205587084804208,
  0.0003860608547172837,
  0.0003706092073583267,
  0.00038246002144859783,
  0.00019586236608421407,
  0.0002559370349437464,
  0.00023864208915534165,
  0.00019847850416255077,
  0.00019197469507232422,
  0.0002014184142673719,
  0.00018811766895408043,
  0.0001790394759825189,
  0.0001740146358251705,
  0.00016016990510920055,
  0.00015766363604534362,
  0.00014903471105729435,
  0.0001586145945004925,
  0.00012915630560227854,
  0.00011500956060865825,
  0.0001166515690446527,
  -9.62916871076359e-06,
  0.00037963676196347933,
  0.0001585